# Imports

Let us first call the Python libraries needed here, and upload the needed data and code.

In [ ]:
# In case: install missing package(s)
# !pip install rhodium 

# Imports
from model import setup, balance_calcs, performance
import rhodium
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Introduction

In this part we will use what we learned from the lake problem to run NSGA-2 with our Conowingo Dam case. Learning to apply a software tool demonstrated in a situation A to the situation B we are interested in is a key engineering skill in today’s world.

The goal here is not to carry out a sophisticated trade-off analysis for the Conowingo Dam, but rather to make sure we know how to run an MOEA with the model we are building. This is why we are focusing on a simple case.

We want to have a policy that restricts withdrawals from the City of Baltimore by a given fraction below a given lake level. We assume that a desalination plant planned to ensure continuous supply. Decision variables are:

1. The fraction reduction in withdrawals from Baltimore (“reduction”, on a 0-1 range)
2. The level at which this happens (“reduction_level”)

In this tutorial, let us keep objectives simple and simply have two:

1. Baltimore volumetric reliability (so different values of “reduction” are reflected differently)
2. Reliability of Peach Bottom nuclear plant (we know that’s the same as volumetric reliability: at the nuclear plant when there’s a shortage it’s because the water level is below the intake: then the shortage is 100%).

# Preparation

In this tutorial we will build on the model we developed in the first two tutorials. Compared with tutorial 2:
* We put new functions to compute reliability and RRV indicators into `model.py`
* We replaced pandas DataFrame with numpy arrays inside the main loop of the water balance method to reduce computing time. This will be important for optimization.
* We have a new function that takes the data (Reservoir and flows) as an implicit argument, and relates the decisions to 

In [ ]:
# Preparing the model
reservoir_name = 'Conowingo'
downstream_demand_names = ['Environmental']
direct_demand_names = ['Baltimore', 'Chester', 'Nuclear plant']

# Loading the model!
conowingo_default = setup.define_reservoir(reservoir_name, downstream_demand_names, direct_demand_names)

# Read flow and demand data. We keep this copy of the data for the simulation of different futures.
flows_default = setup.extract_flows(reservoir=conowingo)

Now we define the planning problem by relating performance to design decisions. Note that here we only simulate the system for 6 years (the driest period) to make it shorter during the tutorial.

In [ ]:
def planning_problem(reduction, reduction_depth):
    """
    This function computes desired performance metrics for the decisions given in arguments.

    Arguments:
    reduction: float on [0, 1] range, the reduction in Baltimore water use 
    reduction_depth: Vector of the depth (one value per month, in m) the amount of water the reservoir can get lower before the restriction kicks in.
                This is a regulation.

    Note that the Reservoir class object and flows pandas DataFrame that are copied are implicit arguments here.

    Outputs are the Baltimore volumetric reliabilities and nuclear plant reliability.

    Note we are focusing on the few dry years in the 1960s to make runtime shorter.
    """

    # Get copies of the data so that there is an untouched original copy
    time_mask = (flows_default.index.year > 1961) & (flows_default.index.year < 1968)
    water_flows = flows_default.iloc[time_mask, :].copy()
    reservoir = copy.deepcopy(conowingo_default)

    # Local variable: number of time steps
    t_total = len(water_flows)

    # Local variable: number of seconds in a day
    n_sec = 86400

    # For computing efficiency: convert flows to numpy arrays outside of time loop

    # Inflows (in m3)
    inflows = water_flows['Total inflows (m3/s)'].to_numpy() * n_sec

    # Initialise downstream demand (in m3 and in numpy array format)
    downstream_demands = balance_calcs.downstream_demand_init(reservoir, water_flows, n_sec)

    # Initialise at-site demands (in m3 and in numpy array format)
    at_site_demands = balance_calcs.local_demand_init(reservoir, water_flows, n_sec)

    # Initialise month numbers
    month = water_flows.index.month

    # Initialise outputs
    # Storage needs to account for initial storage
    storage = np.zeros(t_total + 1)
    storage[0] = reservoir.initial_storage
    # Initialise at-site withdrawals and release as water balance components
    withdrawals = np.zeros((t_total, len(reservoir.demand_on_site)))
    release = np.zeros(t_total)

    # Main loop
    for t in range(t_total):

        # Reduction if initial storage below threshold
        # Compute height of water after environmental flows satisfied
        height = reservoir.get_height(storage[t] + inflows[t] - downstream_demands[t]) 
        # Define withdrawals from reservoir and modify them as needed
        to_withdraw = at_site_demands[t, :]
        if height + reduction_depth[month[t] - 1] < reservoir.total_lake_depth:
            to_withdraw[0] = to_withdraw[0] * (1-reduction)

        # One-step water balance including reduction
        wb_out = balance_calcs.sop_single_step(reservoir, storage[t], inflows[t], to_withdraw, downstream_demands[t])
        storage[t+1] = wb_out[0]
        release[t] = wb_out[1]
        withdrawals[t, :] = wb_out[2]

    # Insert data into water balance (mind the flow rates conversions back into m3/s)
    for i in range(withdrawals.shape[1]):
        water_flows['Withdrawals ' + reservoir.demand_on_site[i].name + ' (m3/s)'] = withdrawals[:, i] / n_sec
    water_flows['Release (m3/s)'] = release / n_sec
    water_flows['Storage (m3)'] = storage[1:]

    # We compute performance indicators
    # Baltimore volumetric reliability
    baltimore_vol_rel = water_flows.sum(axis=0)['Withdrawals Baltimore (m3/s)'] / water_flows.sum(axis=0)['Baltimore demand (m3/s)']
    # Nuclear plant reliability
    nuclear_rel = performance.reliability(water_flows['Withdrawals Nuclear plant (m3/s)'], water_flows['Nuclear plant demand (m3/s)'], True)

    return baltimore_vol_rel, nuclear_rel

In [ ]:
# Note what happens when we call the function
default_perf = planning_problem(0, np.zeros(12))
print('Without intervention, in 1962-1967 Baltimore volumetric reliability is ' + "{:.2f}".format(default_perf[0]))
print('and the nuclear plant reliability is ' + "{:.2f}".format(default_perf[1]))

In [ ]:
# Let us now define the model for the Rhodium library
conowingo_model = rhodium.Model(planning_problem)

conowingo_model.parameters = [rhodium.Parameter('reduction'),
                              rhodium.Parameter('reduction_depth')]

conowingo_model.responses = [rhodium.Response('baltimore_vol_rel', rhodium.Response.MAXIMIZE),
                             rhodium.Response('nuclear_rel', rhodium.Response.MAXIMIZE)]

conowingo_model.levers = [rhodium.RealLever('reduction', 0.0, 1.0),
                          rhodium.RealLever('reduction_depth', 0.0, conowingo_default.demand_on_site[0].intake_depth, length=12)]

In [ ]:
# Optimize
output = rhodium.optimize(conowingo_model, 'NSGAII', 2000)
print("Found", len(output), "optimal policies!")

In [ ]:
print(output)

In [ ]:
# Figures that look fancy in one place sometimes are not so easy to reproduce somewhere else.
# But as long as the errors don't prevent the plotting.
fig = rhodium.parallel_coordinates(conowingo_model, output, c='nuclear_rel')

In [ ]:
# We can save the output in DataFrame format
output_df = output.as_dataframe()
output_df['Large plant'] = (output_df['reduction'] > 0.5)
print(output_df)

# Then we can use pandas tools to plot parallel coordinates
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax = pd.plotting.parallel_coordinates(output_df, 'Large plant')

In [ ]:
# Sometimes there are high values of reduction_depth, so we can also zoom on the [0,1] portion of the y-axis.
ax.set_ylim(0,1)
ax.legend(loc='lower right')
fig

What do we learn?

* We trade-off up to 5% volumetric reliability to Baltimore for less than 0.5% reliability to the nuclear plant. 

* We see the solutions that benefit most 1) Baltimore (reduction close to 0), and 2) Peach Bottom nuclear plant (100% reduction for a full reservoir: Baltimore does not take water out of Conowingo dam anymore unless there is excess water: the reservoir is full and all other demands are met).

* The Pareto Front covers the whole range between the extremes solutions.

* With the exception of extreme solutions, most solutions have restrictions that kick in when the reservoir level gets lower than the maximal level by less than 1m. This is logical given the depth of the Peach Bottom intake: if the reductions only kick in when water level is right above the intake this might have little effect on levels getting lower.

* Broadly, the greater the reduction, the better for Peach Bottom and the worse for Baltimore.
